In [1]:
import pandas as pd
import numpy as np
import os
import re

# Encontrando atos no DODFCorpus_contratos_licitacoes_v0

In [29]:
data = pd.read_csv('Corpus3/DODFCorpus_contratos_licitacoes_v1.csv')

In [30]:
def clean_text_by_word(text):
    a = "\n".join([l for l in text.split("\n") if l != ""])
    words = a.replace("\n", " ").split(" ")
    words = [w for w in words if w != ""]
    
    m_words = []
    dash_cut = False

    for i in range(len(words)):
        word = words[i]

        if (word[-1] == "-") and (i+1)<len(words):
            word = word[:-1] + words[i+1]
            i += 1

        m_words.append(word)
        
    return " ".join(m_words)

In [31]:
data1 = data[['id_ato', 'id_dodf', 'tipo_rel', 'tipo_ent', 'texto']].loc[(data['tipo_ent'] == 'EXTRATO_ADITAMENTO_CONTRATUAL') |\
                                                               (data['tipo_ent'] == 'AVISO_LICITACAO') |\
                                                               (data['tipo_ent'] == 'AVISO_SUSPENSAO_LICITACAO') |\
                                                               (data['tipo_ent'] == 'EXTRATO_CONTRATO') |\
                                                               (data['tipo_ent'] == 'EXTRATO_CONVENIO') |\
                                                               (data['tipo_ent'] == 'AVISO_ANUL_REV_LICITACAO')]

In [32]:
data = data[['id_ato', 'id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel', 
       'tipo_ent', 'id_ent', 'anotador_ent','offset', 'length', 'texto']
       ].loc[(data['tipo_ent'] != 'EXTRATO_ADITAMENTO_CONTRATUAL')]

data = data[['id_ato', 'id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel', 
       'tipo_ent', 'id_ent', 'anotador_ent','offset', 'length', 'texto']
       ].loc[(data['tipo_ent'] != 'AVISO_LICITACAO')]

data = data[['id_ato', 'id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel', 
       'tipo_ent', 'id_ent', 'anotador_ent','offset', 'length', 'texto']
       ].loc[(data['tipo_ent'] != 'AVISO_SUSPENSAO_LICITACAO')]

data = data[['id_ato', 'id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel', 
       'tipo_ent', 'id_ent', 'anotador_ent','offset', 'length', 'texto']
       ].loc[(data['tipo_ent'] != 'EXTRATO_CONTRATO')]

data = data[['id_ato', 'id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel', 
       'tipo_ent', 'id_ent', 'anotador_ent','offset', 'length', 'texto']
       ].loc[(data['tipo_ent'] != 'EXTRATO_CONVENIO')]

data = data[['id_ato', 'id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel', 
       'tipo_ent', 'id_ent', 'anotador_ent','offset', 'length', 'texto']
       ].loc[(data['tipo_ent'] != 'AVISO_ANUL_REV_LICITACAO')]   

In [33]:
data = data.reset_index(drop=True)
data1 = data1.reset_index(drop=True)

In [34]:
tokens = data['tipo_ent'].unique()

for token in tokens:
    data1[token] = np.nan

In [35]:
for i in range(len(data1)):
    texto = clean_text_by_word(data1.loc[i, 'texto']).replace("\r", "")
    data1.loc[i, 'texto'] = texto

In [36]:
for i in range(len(data)):
    for j in range(len(data1)):
        if data['id_ato'][i] == data1['id_ato'][j]:
            entitie = re.sub('xxbcet ?|xxbcet ?|xxeob ?|xxbob ?|xxecet ?', '', clean_text_by_word(data['texto'][i]).replace("\r", ""))
            data1.loc[j, data['tipo_ent'][i]] = entitie

In [37]:
data1['VALOR'] = np.nan

for i in range(len(data1)):
    if type(data1['valor_contrato'][i]) == str:
        data1.loc[i, 'VALOR'] = data1['valor_contrato'][i]
    elif type(data1['valor_convenio'][i]) == str:
        data1.loc[i, 'VALOR'] = data1['valor_convenio'][i]

data1['DATA_ASSINATURA'] = np.nan

for i in range(len(data1)):
    if type(data1['data_assinatura_contrato'][i]) == str:
        data1.loc[i, 'DATA_ASSINATURA'] = data1['data_assinatura_contrato'][i]
    elif type(data1['data_assinatura_convenio'][i]) == str:
        data1.loc[i, 'DATA_ASSINATURA'] = data1['data_assinatura_convenio'][i]

data1['NUM_AJUSTE'] = np.nan

for i in range(len(data1)):
    if type(data1['numero_contrato'][i]) == str:
        data1.loc[i, 'NUM_AJUSTE'] = data1['numero_contrato'][i]
    elif type(data1['numero_convenio'][i]) == str:
        data1.loc[i, 'NUM_AJUSTE'] = data1['numero_convenio'][i]

data1['OBJ_AJUSTE'] = np.nan

for i in range(len(data1)):
    if type(data1['objeto_contrato'][i]) == str:
        data1.loc[i, 'OBJ_AJUSTE'] = data1['objeto_contrato'][i]
    elif type(data1['objeto_convenio'][i]) == str:
        data1.loc[i, 'OBJ_AJUSTE'] = data1['objeto_convenio'][i]

data1['VIGENCIA'] = np.nan

for i in range(len(data1)):
    if type(data1['vigencia_contrato'][i]) == str:
        data1.loc[i, 'VIGENCIA'] = data1['vigencia_contrato'][i]
    elif type(data1['vigencia_convenio'][i]) == str:
        data1.loc[i, 'VIGENCIA'] = data1['vigencia_convenio'][i]

## Renomeando as colunas

In [45]:
data1.columns

Index(['id_ato', 'dodf', 'tipo_rel', 'ato', 'treated_text',
       'MODALIDADE_LICITACAO', 'NUM_LICITACAO', 'VALOR_ESTIMADO',
       'DATA_ABERTURA', 'SISTEMA_COMPRAS', 'NOME_RESPONSAVEL', 'TIPO_OBJ',
       'PROCESSO', 'OBJ_LICITACAO', 'CODIGO_SISTEMA_COMPRAS',
       'ORGAO_LICITANTE', 'DECISAO_TCDF', 'numero_contrato', 'CONTRATANTE',
       'CONTRATADA', 'valor_contrato', 'NOTA_EMPENHO', 'CODIGO_UO',
       'PROGRAMA_TRABALHO', 'NATUREZA_DESPESA', 'FONTE_RECURSO',
       'data_assinatura_contrato', 'objeto_contrato', 'vigencia_contrato',
       'NUM_ADITIVO', 'OBJ_ADITIVO', 'DATA_ESCRITO', 'CNPJ_CONTRATADA',
       'numero_convenio', 'CONCEDENTE', 'CONVENENTE', 'vigencia_convenio',
       'CNPJ_CONVENENTE', 'valor_convenio', 'objeto_convenio',
       'data_assinatura_convenio', 'IDENTIFICACAO_OCORRENCIA',
       'CNPJ_CONTRATANTE', 'CNPJ_CONCEDENTE', 'CODIGO_SIGGO', 'VALOR',
       'DATA_ASSINATURA', 'NUM_AJUSTE', 'OBJ_AJUSTE', 'VIGENCIA'],
      dtype='object')

In [42]:
data1.rename({'id_dodf': 'dodf', 'tipo_ent': 'ato', 'texto': 'treated_text',
       'numero_termo_aditivo': 'NUM_ADITIVO', 'orgao_contratante': 'CONTRATANTE',
       'objeto_aditamento_contratual': 'OBJ_ADITIVO', 'processo_gdf': 'PROCESSO', 'modalidade_licitacao': 'MODALIDADE_LICITACAO',
       'numero_licitacao': 'NUM_LICITACAO', 'orgao_licitante': 'ORGAO_LICITANTE', 'sistema_compras': 'SISTEMA_COMPRAS',
       'valor_estimado_contratacao': 'VALOR_ESTIMADO', 'data_abertura_licitacao': 'DATA_ABERTURA',
       'nome_responsavel': 'NOME_RESPONSAVEL', 'tipo_objeto': 'TIPO_OBJ', 'objeto_licitacao': 'OBJ_LICITACAO', 'data_escrito': 'DATA_ESCRITO',
       'codigo_licitacao_sistema_compras': 'CODIGO_SISTEMA_COMPRAS', 'decisao_tcdf': 'DECISAO_TCDF',
       'entidade_contratada': 'CONTRATADA', 'nota_empenho': 'NOTA_EMPENHO',
       'unidade_orcamentaria': 'CODIGO_UO', 'programa_trabalho': 'PROGRAMA_TRABALHO', 'natureza_despesa': 'NATUREZA_DESPESA',
       'fonte_recurso': 'FONTE_RECURSO',
       'cnpj_entidade_contratada': 'CNPJ_CONTRATADA', 'orgao_concedente': 'CONCEDENTE',
       'entidade_convenente': 'CONVENENTE', 'cnpj_entidade_convenente': 'CNPJ_CONVENENTE',
       'identificacao_ocorrencia': 'IDENTIFICACAO_OCORRENCIA',
       'cnpj_orgao_contratante': 'CNPJ_CONTRATANTE', 'cnpj_orgao_concedente': 'CNPJ_CONCEDENTE', 'codigo_siggo': 'CODIGO_SIGGO'}, axis=1, inplace=True)


In [46]:
data_acts = data1[['ato', 'dodf', 'treated_text', 'PROCESSO',
       'NUM_AJUSTE', 'CONTRATANTE', 'CONTRATADA', 'OBJ_AJUSTE', 'VIGENCIA',
       'VALOR', 'PROGRAMA_TRABALHO', 'DATA_ASSINATURA', 'CODIGO_UO', 'NATUREZA_DESPESA', 'NOTA_EMPENHO',
       'ORGAO_LICITANTE', 'NUM_LICITACAO', 'OBJ_LICITACAO',
       'MODALIDADE_LICITACAO', 'DATA_ABERTURA', 'TIPO_OBJ', 'VALOR_ESTIMADO',
       'CODIGO_SISTEMA_COMPRAS', 'OBJ_ADITIVO', 'NUM_ADITIVO',
       'SISTEMA_COMPRAS', 'CONVENENTE', 'DECISAO_TCDF',
       'NOME_RESPONSAVEL', 'DATA_ESCRITO', 'FONTE_RECURSO',
       'CNPJ_CONTRATADA', 'CONCEDENTE',  'CNPJ_CONVENENTE',
       'IDENTIFICACAO_OCORRENCIA', 'CNPJ_CONTRATANTE', 'CNPJ_CONCEDENTE', 'CODIGO_SIGGO']]

In [47]:
# data_acts.to_csv('Corpus3/data_acts.csv', index=False)

# Separando atos

In [6]:
# data_acts = pd.read_csv('Corpus2/data_acts.csv')

In [48]:
data_acts['ato'].unique()

array(['AVISO_LICITACAO', 'AVISO_SUSPENSAO_LICITACAO', 'EXTRATO_CONTRATO',
       'EXTRATO_ADITAMENTO_CONTRATUAL', 'EXTRATO_CONVENIO',
       'AVISO_ANUL_REV_LICITACAO'], dtype=object)

In [49]:
lic = data_acts[list(data_acts.columns)].loc[(data_acts['ato'] == 'AVISO_LICITACAO')]
lic = lic.reset_index(drop=True)

sus = data_acts[list(data_acts.columns)].loc[(data_acts['ato'] == 'AVISO_SUSPENSAO_LICITACAO')]
sus = sus.reset_index(drop=True)

anr = data_acts[list(data_acts.columns)].loc[(data_acts['ato'] == 'AVISO_ANUL_REV_LICITACAO')]
anr = anr.reset_index(drop=True)

adi = data_acts[list(data_acts.columns)].loc[(data_acts['ato'] == 'EXTRATO_ADITAMENTO_CONTRATUAL')]
adi = adi.reset_index(drop=True)

con = data_acts[list(data_acts.columns)].loc[(data_acts['ato'] == 'EXTRATO_CONTRATO') |\
                               (data_acts['ato'] == 'EXTRATO_CONVENIO') |\
                               (data_acts['ato'] == 'EXTRATO_CONTRATO_CONVENIO')]
con = con.reset_index(drop=True)

In [55]:
len(lic), len(anr), len(sus), len(adi), len(con), len(data_acts)

(638, 46, 68, 1537, 1566, 3855)

In [56]:
ent = ['ato', 'dodf', 'treated_text']

for i in range(3, len(list(lic.isnull().sum()))):
    if list(lic.isnull().sum())[i] != len(lic):
        ent.append(list(lic.columns)[i])

lic = lic[ent]

In [57]:
ent = ['ato', 'dodf', 'treated_text']

for i in range(3, len(list(anr.isnull().sum()))):
    if list(anr.isnull().sum())[i] != len(anr):
        ent.append(list(anr.columns)[i])

anr = anr[ent]

In [58]:
ent = ['ato', 'dodf', 'treated_text']

for i in range(3, len(list(sus.isnull().sum()))):
    if list(sus.isnull().sum())[i] != len(sus):
        ent.append(list(sus.columns)[i])

sus = sus[ent]

In [59]:
ent = ['ato', 'dodf', 'treated_text']

for i in range(3, len(list(adi.isnull().sum()))):
    if list(adi.isnull().sum())[i] != len(adi):
        ent.append(list(adi.columns)[i])

adi = adi[ent]

In [60]:
ent = ['ato', 'dodf', 'treated_text']

for i in range(3, len(list(con.isnull().sum()))):
    if list(con.isnull().sum())[i] != len(con):
        ent.append(list(con.columns)[i])

con = con[ent]

In [61]:
nome = False
for i in range(len(con)):
    if not pd.isna(con['NOME_RESPONSAVEL'][i]):
        nome = True
        break
    
if not nome:
    con['NOME_RESPONSAVEL'] = np.nan

# Unindo contrato e convênio

In [62]:
con['CONTRATANTE_ou_CONCEDENTE'] = np.nan

for i in range(len(con)):
    if not pd.isna(con['CONTRATANTE'][i]):
        con.loc[i, 'CONTRATANTE_ou_CONCEDENTE'] = con['CONTRATANTE'][i]
    elif not pd.isna(con['CONCEDENTE'][i]):
        con.loc[i, 'CONTRATANTE_ou_CONCEDENTE'] = con['CONCEDENTE'][i]

In [63]:
con['CONTRATADA_ou_CONVENENTE'] = np.nan

for i in range(len(con)):
    if not pd.isna(con['CONTRATADA'][i]):
        con.loc[i, 'CONTRATADA_ou_CONVENENTE'] = con['CONTRATADA'][i]
    elif not pd.isna(con['CONVENENTE'][i]):
        con.loc[i, 'CONTRATADA_ou_CONVENENTE'] = con['CONVENENTE'][i]

In [64]:
con['CNPJ_CONTRATANTE_ou_CONCEDENTE'] = np.nan

for i in range(len(con)):
    if not pd.isna(con['CNPJ_CONTRATANTE'][i]):
        con.loc[i, 'CNPJ_CONTRATANTE_ou_CONCEDENTE'] = con['CNPJ_CONTRATANTE'][i]
    elif not pd.isna(con['CNPJ_CONCEDENTE'][i]):
        con.loc[i, 'CNPJ_CONTRATANTE_ou_CONCEDENTE'] = con['CNPJ_CONCEDENTE'][i]

In [65]:
con['CNPJ_CONTRATADA_ou_CONVENENTE'] = np.nan

for i in range(len(con)):
    if not pd.isna(con['CNPJ_CONTRATADA'][i]):
        con.loc[i, 'CNPJ_CONTRATADA_ou_CONVENENTE'] = con['CNPJ_CONTRATADA'][i]
    elif not pd.isna(con['CNPJ_CONVENENTE'][i]):
        con.loc[i, 'CNPJ_CONTRATADA_ou_CONVENENTE'] = con['CNPJ_CONVENENTE'][i]

In [67]:
con = con[['ato', 'dodf', 'treated_text', 'PROCESSO', 'NUM_AJUSTE',
       'OBJ_AJUSTE', 'VIGENCIA', 'VALOR', 'PROGRAMA_TRABALHO',
       'DATA_ASSINATURA', 'CODIGO_UO', 'NATUREZA_DESPESA', 'NOTA_EMPENHO',
       'NOME_RESPONSAVEL', 'FONTE_RECURSO',
       'CONTRATANTE_ou_CONCEDENTE', 'CONTRATADA_ou_CONVENENTE',
       'CNPJ_CONTRATANTE_ou_CONCEDENTE', 'CNPJ_CONTRATADA_ou_CONVENENTE', 'CODIGO_SIGGO']]


## IOB

In [68]:
import nltk
from nltk.tokenize import word_tokenize
from sklearn.base import BaseEstimator, TransformerMixin

In [69]:
class Tokenizer(TransformerMixin, BaseEstimator):
    def __init__(self, tokenizer=''):
        self.tokenizer = tokenizer


    def __call__(self, X, **kw_params):
        return self.tokenizer(X, **kw_params)


    def fit(self, X, y=None, **fit_params):
        return self


    def transform(self, X, **kw_params):
        if not isinstance(X, pd.Series):
            print("[preprocess.Tokenizer.transform] TYPE:", type(X))
            print('X:::: ', X)
            X = pd.Series(X)
        return X.map(self)


from sklearn.base import BaseEstimator, TransformerMixin

class IOBifyer(TransformerMixin, BaseEstimator):

    @staticmethod
    def find_entity(row, token, ignore_idx=0,
        tokenizer=''):
        # TODO: aceitar opção de offset, para não ter tennhum tipo de problema
        for idx, column in enumerate(row.keys()):
            if idx == ignore_idx:
                continue
            if isinstance(row[column], str) and \
                token == word_tokenize(row[column])[0]:
                return column

        return None


    @staticmethod
    def generate_IOB_labels(row, idx, tokenizer, dbg={}):
        labels = []
        entity_started = False
        text = row.iloc[idx]
        for token in word_tokenize(text):                         
            if not entity_started:                               
                entity = IOBifyer.find_entity(row, token, idx)                 
                if entity is not None:                           
                    entity_started = True
                    token_index = 1
                    labels.append('B-' + entity)
                else:
                    labels.append('O')
            else:
                if token_index < len(word_tokenize(row[entity])) and \
                    token == word_tokenize(row[entity])[token_index]:
                    labels.append('I-' + entity)
                    token_index += 1
                    if token_index >= len(word_tokenize(row[entity])):
                        entity_started = False
                else:
                    entity_started = False
                    labels.append('O')
        if labels[0] != 'O':
            dbg['l'] = dbg.get('l', []) + [(row, idx)]

        return labels


    @staticmethod
    def dump_iob(tokens_mat, labels_mat, path='dump.txt',
                            sep=' X X ', sent_sep='\n',):
        dbg_mat = []
        if isinstance(path, Path):
            path = path.as_posix()
        if '/' in path:
            os.makedirs('/'.join(path.split('/')[:-1]), exist_ok=True)

        with open(path, 'w') as fp:
            for tokens_lis, labels_lis in zip(tokens_mat, labels_mat):
                dbg_mat.append([])
                for token, label in zip(tokens_lis, labels_lis):
                    dbg_mat[-1].append((token, label))
                    fp.write(f"{token}{sep}{label}\n")
                fp.write(sent_sep)
        return dbg_mat


    def __init__(self, column='act_column',
        tokenizer=''):
        self.column = column
        self.tokenizer = tokenizer
        self.dbg = {}


    def fit(self, X=None, y=None, **fit_params):
        return self


    def transform(self, df):
        if not isinstance(df, pd.DataFrame):
            raise TypeError(f"`df` expected to be a pd.DataFrame. Got {type(df)}")
        if df.empty:
            print("[core.preprocess]Warning: empty DataFrame. There won't be ioblabels.")
            return pd.Series()

        idx = self.column if isinstance(self.column, int) else  \
                df.columns.get_loc(self.column)
        labels_row = []
        for index, row in df.iterrows():
            try:
                labels_row.append(
                    IOBifyer.generate_IOB_labels(
                        row, idx, self.tokenizer, self.dbg
                    )
                )
            except Exception as e:
                print("problem iobifyin row:", row)
                raise e
        return pd.Series(labels_row)


In [72]:
# Licitacao
lic_iob = lic[list(lic.columns)[2:]]

iob_lic= IOBifyer(column='treated_text')
r_lic = iob_lic.transform(lic_iob)
lic["IOB"] = np.nan

for i in range(len(lic)):
    lic.loc[i, "IOB"] = ' '.join(r_lic[i])

C:\Users\ianfi\AppData\Local\Temp\ipykernel_20136\1078628249.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lic["IOB"] = np.nan
C:\Users\ianfi\AppData\Local\Temp\ipykernel_20136\1078628249.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lic.loc[i, "IOB"] = ' '.join(r_lic[i])


In [77]:
for i in range(len(lic)):
    if len(lic.loc[0, "IOB"].split()) != len(word_tokenize(lic.loc[0, "treated_text"])):
        print(i)

In [78]:
# Suspensao
sus_iob = sus[list(sus.columns)[2:]]

iob_sus = IOBifyer(column='treated_text')
r_sus = iob_sus.transform(sus_iob)
sus["IOB"] = np.nan

for i in range(len(sus)):
    sus.loc[i, "IOB"] = ' '.join(r_sus[i])

In [79]:
for i in range(len(sus)):
    if len(sus.loc[0, "IOB"].split()) != len(word_tokenize(sus.loc[0, "treated_text"])):
        print(i)

In [80]:
# Anulacao e revogacao
anr_iob = anr[list(anr.columns)[2:]]

iob_anr = IOBifyer(column='treated_text')
r_anr = iob_anr.transform(anr_iob)
anr["IOB"] = np.nan

for i in range(len(anr)):
    anr.loc[i, "IOB"] = ' '.join(r_anr[i])

In [81]:
for i in range(len(anr)):
    if len(anr.loc[0, "IOB"].split()) != len(word_tokenize(anr.loc[0, "treated_text"])):
        print(i)

In [83]:
# Aditamento
adi_iob = adi[list(adi.columns)[2:]]

iob_adi = IOBifyer(column='treated_text')
r_adi = iob_adi.transform(adi_iob)
adi["IOB"] = np.nan

for i in range(len(adi)):
    adi.loc[i, "IOB"] = ' '.join(r_adi[i])

In [84]:
for i in range(len(adi)):
    if len(adi.loc[0, "IOB"].split()) != len(word_tokenize(adi.loc[0, "treated_text"])):
        print(i)

In [85]:
# Contrato e convenio
con_iob = con[list(con.columns)[2:]]

iob_con = IOBifyer(column='treated_text')
r_con = iob_con.transform(con_iob)
con["IOB"] = np.nan

for i in range(len(con)):
    con.loc[i, "IOB"] = ' '.join(r_con[i])

In [86]:
for i in range(len(con)):
    if len(con.loc[0, "IOB"].split()) != len(word_tokenize(con.loc[0, "treated_text"])):
        print(i)

In [87]:
len(lic), len(sus), len(anr), len(adi), len(con)

(638, 68, 46, 1537, 1566)

In [88]:
# lic.to_csv('Corpus3/licitacao.csv', index=False)
# sus.to_csv('Corpus3/suspensao.csv', index=False)
# anr.to_csv('Corpus3/anulacao_revogacao.csv', index=False)
# adi.to_csv('Corpus3/aditamento_contratual.csv', index=False)
# con.to_csv('Corpus3/contrato_convenio.csv', index=False)